In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal
from utils import device, gaussian_grid, get_coords

In [9]:
res = 400

mu = torch.tensor([0.3, -0.5], device=device)
cov = torch.tensor([
    [0.05, -0.03],
    [-0.03, 0.05]
], device=device)
cov_ = torch.inverse(cov)

In [10]:
x = torch.linspace(-1, 1, res, device=device)
x = torch.meshgrid(x, x, indexing='ij')
x = torch.stack(x).view(2, -1).T

In [11]:
pred = (torch.e ** (-(1 / 2) * ((x - mu) * ((x - mu) @ cov_.T)).sum(1)))

plt.contourf(pred.view(res, res).T.cpu(), levels=100)
plt.show()

1.15 ms ± 44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
